In [27]:
"""
========================================
FAIR BASELINE BENCHMARK - Q1 TIME SERIES
========================================
All models constrained to ~20,000 parameters
Fast execution on Kaggle P100 / CUDA
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import time
import pickle
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# ==============================================================================
# CONFIGURATION
# ==============================================================================
CONFIG = {
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu',
    'BATCH_SIZE': 2048, # Large batch for speed on simple data
    'EPOCHS': 20,
    'LR': 0.003,
    'SEQ_LEN': 96,
    'PRED_LEN': 24,
    'TARGET_PARAMS': 20000, 
    'SEED': 42,
    'SAVE_DIR': Path('./benchmark_results')
}

CONFIG['SAVE_DIR'].mkdir(exist_ok=True)
torch.manual_seed(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])

print(f"🚀 Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"🎯 Target: ~{CONFIG['TARGET_PARAMS']:,} parameters per model\n")

# ==============================================================================
# DATASETS
# ==============================================================================

def load_electricity(n=15000):
    print("Loading Electricity...", end=' ')
    t = np.arange(n)
    data = np.zeros((n, 12))
    for i in range(12):
        daily = np.sin(t * 2*np.pi / 24 + i*0.5)
        weekly = 1.5 * np.sin(t * 2*np.pi / 168)
        trend = np.linspace(0, 2, n)
        noise = np.random.normal(0, 0.3, n)
        data[:, i] = daily + weekly + trend + noise
    scaler = StandardScaler()
    print("✓")
    return scaler.fit_transform(data).astype(np.float32)

def load_traffic(n=15000):
    # Function kept for structure, but not used in datasets dictionary
    print("Loading Traffic...", end=' ')
    t = np.arange(n)
    data = np.zeros((n, 8))
    for i in range(8):
        morning = 2 * np.exp(-((t % 24 - 8)**2) / 8)
        evening = 2 * np.exp(-((t % 24 - 18)**2) / 8)
        weekend = 0.5 * np.sin(t * 2*np.pi / 168)
        baseline = 1 + 0.3 * np.sin(t * 2*np.pi / (24*365))
        noise = np.random.gamma(2, 0.2, n)
        data[:, i] = baseline * (1 + morning + evening) * (1 + weekend) + noise
    scaler = StandardScaler()
    print("✓")
    return scaler.fit_transform(data).astype(np.float32)

def create_dataset(data, seq_len, pred_len):
    X, y = [], []
    for i in range(len(data) - seq_len - pred_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len:i+seq_len+pred_len, 0]) # Target: First column only
    
    X, y = np.array(X), np.array(y)
    split = int(0.7 * len(X))
    val_split = int(0.85 * len(X))
    
    return (
        (X[:split], y[:split]),
        (X[split:val_split], y[split:val_split]),
        (X[val_split:], y[val_split:])
    )

# ETT removed from here
# Traffic removed from here as requested
datasets = {
    'Electricity': load_electricity()
}

# ==============================================================================
# MODELS (~20k params)
# ==============================================================================

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class LSTM_Model(nn.Module):
    def __init__(self, input_dim, pred_len=24):
        super().__init__()
        # Tuned to ~20k
        hidden = int(np.sqrt(20000 / (4 * input_dim + 1)))
        self.lstm = nn.LSTM(input_dim, hidden, 2, batch_first=True, dropout=0.1)
        self.fc = nn.Linear(hidden, pred_len)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

class GRU_Model(nn.Module):
    def __init__(self, input_dim, pred_len=24):
        super().__init__()
        hidden = int(np.sqrt(20000 / (3 * input_dim + 1)))
        self.gru = nn.GRU(input_dim, hidden, 2, batch_first=True, dropout=0.1)
        self.fc = nn.Linear(hidden, pred_len)
    
    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])

class Transformer_Model(nn.Module):
    def __init__(self, input_dim, pred_len=24):
        super().__init__()
        d_model = 32
        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_encoder = nn.Parameter(torch.randn(1, 500, d_model) * 0.02)
        
        layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=4, dim_feedforward=d_model*2,
            dropout=0.1, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(layer, num_layers=2)
        self.fc = nn.Linear(d_model, pred_len)
    
    def forward(self, x):
        x = self.input_proj(x) + self.pos_encoder[:, :x.size(1), :]
        x = self.transformer(x)
        return self.fc(x[:, -1, :])

class PatchTST_Model(nn.Module):
    def __init__(self, input_dim, seq_len=96, pred_len=24):
        super().__init__()
        patch_len = 16
        self.patch_len = patch_len
        self.num_patches = seq_len // patch_len
        
        d_model = 32
        self.patch_embed = nn.Linear(patch_len * input_dim, d_model)
        self.pos_embed = nn.Parameter(torch.randn(1, self.num_patches, d_model) * 0.02)
        
        layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=4, dim_feedforward=d_model*2,
            dropout=0.1, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(layer, num_layers=1)
        self.head = nn.Linear(d_model * self.num_patches, pred_len)
    
    def forward(self, x):
        B, T, F = x.shape
        x = x.reshape(B, self.num_patches, self.patch_len * F)
        x = self.patch_embed(x) + self.pos_embed
        x = self.transformer(x)
        return self.head(x.reshape(B, -1))

class FEDformer_Model(nn.Module):
    def __init__(self, input_dim, pred_len=24):
        super().__init__()
        d_model = 32
        self.input_proj = nn.Linear(input_dim, d_model)
        self.freq_attn = nn.MultiheadAttention(d_model, num_heads=4, batch_first=True)
        self.seasonal = nn.Linear(d_model, pred_len)
        self.trend = nn.Linear(d_model, pred_len)
    
    def forward(self, x):
        x = self.input_proj(x)
        x_freq, _ = self.freq_attn(x, x, x)
        return self.seasonal(x_freq[:, -1, :]) + self.trend(x[:, -1, :])

# --- NEW MODELS ADDED BELOW ---

class TSMixer_Model(nn.Module):
    def __init__(self, input_dim, seq_len=96, pred_len=24):
        super().__init__()
        # Tuned to fit ~20k
        d_model = 16
        self.input_lin = nn.Linear(input_dim, d_model)
        
        # 2 Mixing Layers
        self.time_mix = nn.Sequential(
            nn.Linear(seq_len, seq_len),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        self.feature_mix = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        
        # Projection for first column output
        self.head = nn.Linear(seq_len * d_model, pred_len)

    def forward(self, x):
        # x: [B, S, C]
        x = self.input_lin(x) # [B, S, d]
        
        # Time Mixing
        x_t = x.permute(0, 2, 1) # [B, d, S]
        x_t = self.time_mix(x_t)
        x = x + x_t.permute(0, 2, 1)
        
        # Feature Mixing
        x_f = self.feature_mix(x)
        x = x + x_f
        
        # Flatten and Project to (B, Pred)
        x = x.reshape(x.size(0), -1)
        return self.head(x)

class iTransformer_Model(nn.Module):
    def __init__(self, input_dim, seq_len=96, pred_len=24):
        super().__init__()
        d_model = 32 # Embedding entire time series
        self.embed = nn.Linear(seq_len, d_model)
        
        layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=4, dim_feedforward=64,
            dropout=0.1, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(layer, num_layers=1)
        
        # Project channel 0 embedding to prediction
        self.head = nn.Linear(d_model, pred_len)
        
    def forward(self, x):
        # x: [B, S, C] -> Invert -> [B, C, S]
        x = x.permute(0, 2, 1)
        
        # Embed time dim
        x = self.embed(x) # [B, C, d_model]
        
        # Attention across variables
        x = self.encoder(x)
        
        # We only want to predict the first variable (Column 0)
        # So we take the 0th token
        x_target = x[:, 0, :] # [B, d_model]
        
        return self.head(x_target)

class MTGNN_Model(nn.Module):
    def __init__(self, input_dim, seq_len=96, pred_len=24):
        super().__init__()
        # Simplified Graph Convolution for strict parameter budget
        c_out = 8
        self.node_embeddings = nn.Parameter(torch.randn(input_dim, 4), requires_grad=True)
        
        self.gcn_lin = nn.Linear(c_out, c_out)
        self.input_conv = nn.Conv1d(input_dim, c_out, kernel_size=1)
        self.temporal_conv = nn.Conv1d(c_out, c_out, kernel_size=3, padding=1)
        
        # Output: Project from (c_out * seq_len) -> pred_len
        self.head = nn.Linear(c_out * seq_len, pred_len)

    def forward(self, x):
        B, S, C = x.shape
        
        # 1. Learn Graph
        adj = F.softmax(F.relu(torch.mm(self.node_embeddings, self.node_embeddings.T)), dim=1)
        
        # 2. Input Proj: [B, C, S] -> [B, 8, S]
        # (Mixing channels into latent dims)
        x_in = x.permute(0, 2, 1)
        x_conv = self.input_conv(x_in) 
        
        # 3. Graph Conv (Simplified)
        # Usually GCN is (AXW). Here we approximate by mixing latent features
        # We assume latent features capture node info.
        # x_conv: [B, 8, S]
        x_gcn = x_conv.permute(0, 2, 1) # [B, S, 8]
        x_gcn = self.gcn_lin(x_gcn)     # [B, S, 8]
        x_gcn = x_gcn.permute(0, 2, 1)  # [B, 8, S]
        
        # 4. Temporal Conv
        x_out = F.relu(self.temporal_conv(x_conv + x_gcn))
        
        # 5. Output Head
        x_flat = x_out.reshape(B, -1)
        return self.head(x_flat)

# ==============================================================================
# TRAINING ENGINE
# ==============================================================================

def train_model(model, train_loader, val_loader, epochs, lr, device):
    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.MSELoss()
    scaler = torch.cuda.amp.GradScaler()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
    
    best_val_loss = float('inf')
    patience = 0
    start_time = time.time()
    
    for epoch in range(epochs):
        model.train()
        for bx, by in train_loader:
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                pred = model(bx)
                loss = criterion(pred, by)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for bx, by in val_loader:
                with torch.cuda.amp.autocast():
                    pred = model(bx)
                    loss = criterion(pred, by)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience = 0
        else:
            patience += 1
            if patience >= 5:
                break
    
    return model, time.time() - start_time

def evaluate_model(model, test_loader, device):
    model.eval()
    preds, targets = [], []
    
    start = time.time()
    with torch.no_grad():
        for bx, by in test_loader:
            with torch.cuda.amp.autocast():
                pred = model(bx)
            preds.append(pred.cpu().numpy())
            targets.append(by.cpu().numpy())
    
    inference_time = time.time() - start
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    
    mse = mean_squared_error(targets, preds)
    mae = mean_absolute_error(targets, preds)
    rmse = np.sqrt(mse)
    
    return {
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'Inference_Time': inference_time,
        'predictions': preds[:500]
    }

# ==============================================================================
# BENCHMARK
# ==============================================================================

def run_benchmark():
    results = []
    
    model_configs = {
        'LSTM': LSTM_Model,
        'GRU': GRU_Model,
        'Transformer': Transformer_Model,
        'PatchTST': PatchTST_Model,
        'FEDformer': FEDformer_Model,
        'TSMixer': TSMixer_Model,
        'iTransformer': iTransformer_Model,
        'MTGNN': MTGNN_Model
    }
    
    for dataset_name, data in datasets.items():
        print(f"\n{'='*70}")
        print(f"📊 {dataset_name} | Shape: {data.shape}")
        print(f"{'='*70}")
        
        (train_x, train_y), (val_x, val_y), (test_x, test_y) = create_dataset(
            data, CONFIG['SEQ_LEN'], CONFIG['PRED_LEN']
        )
        
        train_loader = DataLoader(
            TensorDataset(
                torch.tensor(train_x, device=CONFIG['DEVICE']),
                torch.tensor(train_y, device=CONFIG['DEVICE'])
            ),
            batch_size=CONFIG['BATCH_SIZE'], shuffle=True
        )
        
        val_loader = DataLoader(
            TensorDataset(
                torch.tensor(val_x, device=CONFIG['DEVICE']),
                torch.tensor(val_y, device=CONFIG['DEVICE'])
            ),
            batch_size=CONFIG['BATCH_SIZE'] * 2
        )
        
        test_loader = DataLoader(
            TensorDataset(
                torch.tensor(test_x, device=CONFIG['DEVICE']),
                torch.tensor(test_y, device=CONFIG['DEVICE'])
            ),
            batch_size=CONFIG['BATCH_SIZE'] * 2
        )
        
        # Save test data
        with open(CONFIG['SAVE_DIR'] / f'{dataset_name}_test.pkl', 'wb') as f:
            pickle.dump({
                'train_x': train_x, 'train_y': train_y,
                'val_x': val_x, 'val_y': val_y,
                'test_x': test_x, 'test_y': test_y,
                'input_dim': data.shape[1]
            }, f)
        
        for model_name, model_class in model_configs.items():
            print(f"⚡ {model_name:12s} ", end='')
            
            try:
                # Instantiate with correct args based on class signature
                if model_name in ['DLinear', 'MTGNN']:
                    model = model_class(data.shape[1], CONFIG['SEQ_LEN'], CONFIG['PRED_LEN'])
                elif model_name in ['PatchTST', 'TSMixer', 'iTransformer']:
                    model = model_class(data.shape[1], CONFIG['SEQ_LEN'], CONFIG['PRED_LEN'])
                else:
                    # LSTM, GRU, Transformer, FEDformer (standard signature)
                    model = model_class(data.shape[1], CONFIG['PRED_LEN'])
                
                params = count_parameters(model)
                
                model, train_time = train_model(
                    model, train_loader, val_loader,
                    CONFIG['EPOCHS'], CONFIG['LR'], CONFIG['DEVICE']
                )
                
                metrics = evaluate_model(model, test_loader, CONFIG['DEVICE'])
                
                results.append({
                    'Dataset': dataset_name,
                    'Model': model_name,
                    'Params': params,
                    'Train_Time': train_time,
                    **metrics
                })
                
                print(f"✓ MSE={metrics['MSE']:.4f} | Params={params:,} | Time={train_time:.1f}s")
                
                del model
                torch.cuda.empty_cache()
                
            except Exception as e:
                print(f"✗ Error: {str(e)}")
                import traceback
                traceback.print_exc()
    
    # Save
    df = pd.DataFrame(results)
    df.to_csv(CONFIG['SAVE_DIR'] / 'baseline_results.csv', index=False)
    
    with open(CONFIG['SAVE_DIR'] / 'baseline_results.pkl', 'wb') as f:
        pickle.dump(results, f)
    
    # Print summary
    print(f"\n{'='*100}")
    print(f"{'DATASET':<12} | {'MODEL':<12} | {'PARAMS':<8} | {'MSE':<10} | {'MAE':<10} | {'TRAIN(s)':<10}")
    print(f"{'='*100}")
    
    if not df.empty:
        for _, row in df.iterrows():
            print(f"{row['Dataset']:<12} | {row['Model']:<12} | {row['Params']:<8} | "
                  f"{row['MSE']:<10.4f} | {row['MAE']:<10.4f} | {row['Train_Time']:<10.1f}")
    
    print(f"{'='*100}\n")
    
    # Best per dataset
    print("🏆 BEST MODELS:")
    if not df.empty:
        for dataset in df['Dataset'].unique():
            best = df[df['Dataset'] == dataset].nsmallest(1, 'MSE').iloc[0]
            print(f"  {dataset:12s}: {best['Model']:12s} (MSE={best['MSE']:.4f})")
    
    print(f"\n✅ Saved to {CONFIG['SAVE_DIR']}/baseline_results.csv")
    
    return df

if __name__ == '__main__':
    run_benchmark()

🚀 Device: Tesla P100-PCIE-16GB
🎯 Target: ~20,000 parameters per model

Loading Electricity... ✓

📊 Electricity | Shape: (15000, 12)
⚡ LSTM         ✓ MSE=0.1895 | Params=6,584 | Time=4.1s
⚡ GRU          ✓ MSE=0.1400 | Params=6,441 | Time=5.1s
⚡ Transformer  ✓ MSE=0.1020 | Params=34,296 | Time=16.6s
⚡ PatchTST     ✓ MSE=0.0632 | Params=19,544 | Time=3.8s
⚡ FEDformer    ✓ MSE=0.4025 | Params=6,224 | Time=7.4s
⚡ TSMixer      ✓ MSE=0.0515 | Params=46,680 | Time=3.1s
⚡ iTransformer ✓ MSE=0.0740 | Params=12,440 | Time=3.9s
⚡ MTGNN        ✓ MSE=0.0539 | Params=18,880 | Time=5.8s

DATASET      | MODEL        | PARAMS   | MSE        | MAE        | TRAIN(s)  
Electricity  | LSTM         | 6584     | 0.1895     | 0.3543     | 4.1       
Electricity  | GRU          | 6441     | 0.1400     | 0.3015     | 5.1       
Electricity  | Transformer  | 34296    | 0.1020     | 0.2568     | 16.6      
Electricity  | PatchTST     | 19544    | 0.0632     | 0.1999     | 3.8       
Electricity  | FEDformer    | 6

In [28]:
"""
CausalGraphMemoryNet (CGMN) - Time Series Forecasting
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# ==============================================================================
# CONFIGURATION
# ==============================================================================
CONFIG = {
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu',
    'SEQ_LEN': 96,
    'PRED_LEN': 24,
    'PATCH_SIZE': 16,
    'SEEDS': [42, 123, 456],
    'BATCH_SIZE': 512,
    'EPOCHS': 100,
    'LR': 0.002,
    'HIDDEN_DIM': 32,
    'N_HEADS': 4,
    'N_LAYERS': 2,
    'N_MEMORY': 8,
    'GRAPH_DIM': 16,
    'DROPOUT': 0.15,
}

BASELINE = {'mse': 0.0515}

# ==============================================================================
# DATA
# ==============================================================================
def load_electricity(n=15000, seed=42):
    np.random.seed(seed)
    t = np.arange(n)
    data = np.zeros((n, 12))
    for i in range(12):
        phase = i * 0.5
        neighbor = 0.3 * np.sin(t * 2*np.pi / 24 + (i+1) % 12 * 0.5)
        data[:, i] = (np.sin(t * 2*np.pi / 24 + phase) + 
                     1.5*np.sin(t * 2*np.pi / 168) + neighbor +
                     np.linspace(0, 2, n) + np.random.normal(0, 0.3, n))
    return StandardScaler().fit_transform(data).astype(np.float32)

def create_dataset(data, seq_len, pred_len):
    X, y = [], []
    for i in range(len(data) - seq_len - pred_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len:i+seq_len+pred_len, 0])
    X, y = np.array(X), np.array(y)
    s1, s2 = int(0.7*len(X)), int(0.85*len(X))
    return {
        'train_x': X[:s1], 'train_y': y[:s1],
        'val_x': X[s1:s2], 'val_y': y[s1:s2],
        'test_x': X[s2:], 'test_y': y[s2:],
        'input_dim': data.shape[1]
    }

def create_loaders(data, config):
    device = config['DEVICE']
    return {
        'train': DataLoader(
            TensorDataset(
                torch.tensor(data['train_x'], device=device).float(),
                torch.tensor(data['train_y'], device=device).float()
            ), batch_size=config['BATCH_SIZE'], shuffle=True, drop_last=True
        ),
        'val': DataLoader(
            TensorDataset(
                torch.tensor(data['val_x'], device=device).float(),
                torch.tensor(data['val_y'], device=device).float()
            ), batch_size=config['BATCH_SIZE']*2
        ),
        'test': DataLoader(
            TensorDataset(
                torch.tensor(data['test_x'], device=device).float(),
                torch.tensor(data['test_y'], device=device).float()
            ), batch_size=config['BATCH_SIZE']*2
        ),
    }

# ==============================================================================
# COMPONENT 1: Graph-Gated Channel Fusion (GGCF)
# ==============================================================================
class GraphGatedChannelFusion(nn.Module):
    def __init__(self, n_channels, hidden_dim, graph_dim=16):
        super().__init__()
        self.node_src = nn.Parameter(torch.randn(n_channels, graph_dim) * 0.1)
        self.node_tgt = nn.Parameter(torch.randn(n_channels, graph_dim) * 0.1)
        self.temp = nn.Parameter(torch.tensor(1.0))
        
        self.gate = nn.Sequential(
            nn.Linear(n_channels * 2, n_channels),
            nn.Sigmoid()
        )
        self.mix = nn.Sequential(
            nn.Linear(n_channels, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, n_channels)
        )
        self.register_buffer('adj_matrix', torch.zeros(n_channels, n_channels))
    
    def forward(self, x):
        adj = F.softmax(self.node_src @ self.node_tgt.T / (0.5 + torch.sigmoid(self.temp)), dim=-1)
        self.adj_matrix = adj.detach()
        
        x_graph = torch.einsum('btc,cd->btd', x, adj)
        gate = self.gate(torch.cat([x, x_graph], dim=-1))
        return x + gate * self.mix(x_graph)

# ==============================================================================
# COMPONENT 2: Temporal Memory Bank (TMB)
# ==============================================================================
class TemporalMemoryBank(nn.Module):
    def __init__(self, dim, n_memory=8, n_heads=4):
        super().__init__()
        self.n_memory = n_memory
        self.n_heads = n_heads
        self.head_dim = dim // n_heads
        self.scale = self.head_dim ** -0.5
        
        self.memory = nn.Parameter(torch.randn(n_memory, dim) * 0.02)
        self.qkv = nn.Linear(dim, dim * 3)
        self.mem_kv = nn.Linear(dim, dim * 2)
        self.out = nn.Linear(dim, dim)
        self.register_buffer('usage', torch.zeros(n_memory))
    
    def forward(self, x):
        B, N, D = x.shape
        
        qkv = self.qkv(x).reshape(B, N, 3, self.n_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        Q, K, V = qkv[0], qkv[1], qkv[2]
        
        mem = self.memory.unsqueeze(0).expand(B, -1, -1)
        mem_kv = self.mem_kv(mem).reshape(B, self.n_memory, 2, self.n_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        K_mem, V_mem = mem_kv[0], mem_kv[1]
        
        K_full = torch.cat([K_mem, K], dim=2)
        V_full = torch.cat([V_mem, V], dim=2)
        
        attn = (Q @ K_full.transpose(-2, -1)) * self.scale
        
        causal = torch.triu(torch.ones(N, N, device=x.device), 1).bool()
        mask = torch.zeros(N, self.n_memory + N, device=x.device).bool()
        mask[:, self.n_memory:] = causal
        attn = attn.masked_fill(mask.unsqueeze(0).unsqueeze(0), torch.finfo(attn.dtype).min)
        attn = F.softmax(attn, dim=-1)
        
        if self.training:
            with torch.no_grad():
                self.usage = 0.9 * self.usage + 0.1 * attn[:, :, :, :self.n_memory].mean(dim=(0,1,2))
        
        out = (attn @ V_full).transpose(1, 2).reshape(B, N, D)
        return self.out(out)

# ==============================================================================
# COMPONENT 3: Progressive Weight Sharing (PWS)
# ==============================================================================
class SharedWeightFFN(nn.Module):
    def __init__(self, dim, expansion=4, n_templates=4, dropout=0.1):
        super().__init__()
        self.dim = dim
        self.hidden = dim * expansion
        self.n_templates = n_templates
        
        self.templates = nn.Parameter(torch.randn(n_templates, dim, dim) * 0.02)
        self.selector = nn.Linear(dim, n_templates)
        
        self.fc1 = nn.Linear(dim, self.hidden)
        self.fc2 = nn.Linear(self.hidden, dim)
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(dim)
        
        self.register_buffer('sharing_stats', torch.zeros(n_templates))
    
    def forward(self, x):
        weights = F.softmax(self.selector(x.mean(dim=1)), dim=-1)
        
        if self.training:
            with torch.no_grad():
                self.sharing_stats = 0.9 * self.sharing_stats + 0.1 * weights.mean(0)
        
        shared = torch.einsum('bn,nde,bte->btd', weights, self.templates, x)
        
        h = F.gelu(self.fc1(x + shared))
        out = self.fc2(self.dropout(h))
        return self.norm(out)
    
    def get_sharing_ratio(self):
        stats = self.sharing_stats.cpu()
        entropy = -(stats * (stats + 1e-8).log()).sum()
        return (1 - entropy / np.log(self.n_templates)).item()

# ==============================================================================
# ENCODER LAYER
# ==============================================================================
class EncoderLayer(nn.Module):
    def __init__(self, dim, n_memory=8, n_heads=4, dropout=0.1, use_memory=True, use_sharing=True):
        super().__init__()
        self.use_memory = use_memory
        self.use_sharing = use_sharing
        
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(dropout)
        
        if use_memory:
            self.attn = TemporalMemoryBank(dim, n_memory, n_heads)
        else:
            self.attn = nn.MultiheadAttention(dim, n_heads, dropout, batch_first=True)
            self.register_buffer('causal_mask', torch.triu(torch.ones(96//16, 96//16), 1).bool())
        
        if use_sharing:
            self.ffn = SharedWeightFFN(dim, 4, 4, dropout)
        else:
            self.ffn = nn.Sequential(
                nn.Linear(dim, dim * 4),
                nn.GELU(),
                nn.Dropout(dropout),
                nn.Linear(dim * 4, dim),
                nn.LayerNorm(dim)
            )
    
    def forward(self, x):
        if self.use_memory:
            x = x + self.dropout(self.attn(self.norm1(x)))
        else:
            x_norm = self.norm1(x)
            attn_out, _ = self.attn(x_norm, x_norm, x_norm, attn_mask=self.causal_mask)
            x = x + self.dropout(attn_out)
        
        x = x + self.dropout(self.ffn(self.norm2(x)))
        return x

# ==============================================================================
# MAIN MODEL
# ==============================================================================
class CausalGraphMemoryNet(nn.Module):
    def __init__(self, input_dim, seq_len, pred_len, hidden_dim=32, 
                 n_heads=4, n_layers=2, patch_size=16, n_memory=8,
                 graph_dim=16, dropout=0.1,
                 use_graph=True, use_memory=True, use_sharing=True):
        super().__init__()
        
        self.n_patches = seq_len // patch_size
        self.patch_size = patch_size
        self.use_graph = use_graph
        self.use_memory = use_memory
        self.use_sharing = use_sharing
        
        self.register_buffer('eps', torch.tensor(1e-5))
        
        if use_graph:
            self.graph = GraphGatedChannelFusion(input_dim, hidden_dim, graph_dim)
        
        self.patch_embed = nn.Linear(patch_size * input_dim, hidden_dim)
        self.pos_embed = nn.Parameter(torch.randn(1, self.n_patches, hidden_dim) * 0.02)
        
        self.layers = nn.ModuleList([
            EncoderLayer(hidden_dim, n_memory, n_heads, dropout, use_memory, use_sharing)
            for _ in range(n_layers)
        ])
        
        self.norm = nn.LayerNorm(hidden_dim)
        self.head = nn.Linear(hidden_dim * self.n_patches, pred_len)
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.trunc_normal_(m.weight, std=0.02)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x):
        B, T, C = x.shape
        
        self._mean = x.mean(dim=1, keepdim=True)
        self._std = x.std(dim=1, keepdim=True) + self.eps
        x = (x - self._mean) / self._std
        
        if self.use_graph:
            x = self.graph(x)
        
        x = x[:, :self.n_patches * self.patch_size].reshape(B, self.n_patches, -1)
        x = self.patch_embed(x) + self.pos_embed
        
        for layer in self.layers:
            x = layer(x)
        
        x = self.norm(x).reshape(B, -1)
        pred = self.head(x)
        
        return pred * self._std[:, 0, 0:1] + self._mean[:, 0, 0:1]
    
    def get_stats(self):
        stats = {'params': sum(p.numel() for p in self.parameters())}
        if self.use_graph:
            adj = self.graph.adj_matrix.cpu()
            stats['graph_sparsity'] = (adj < 0.1).float().mean().item()
        if self.use_memory:
            stats['memory_util'] = np.mean([l.attn.usage.cpu().mean().item() for l in self.layers])
        if self.use_sharing:
            ratios = [l.ffn.get_sharing_ratio() for l in self.layers if hasattr(l.ffn, 'get_sharing_ratio')]
            stats['sharing'] = np.mean(ratios) if ratios else 0
        return stats

# ==============================================================================
# TRAINING
# ==============================================================================
def train_model(model, train_loader, val_loader, config, verbose=True):
    device = config['DEVICE']
    model.to(device)
    
    optimizer = optim.AdamW(model.parameters(), lr=config['LR'], weight_decay=0.01)
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=config['LR'], epochs=config['EPOCHS'],
        steps_per_epoch=len(train_loader), pct_start=0.1
    )
    
    scaler = torch.cuda.amp.GradScaler()
    best_val, best_state, patience = float('inf'), None, 0
    
    for epoch in range(config['EPOCHS']):
        model.train()
        train_loss = 0
        for xb, yb in train_loader:
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast():
                loss = F.mse_loss(model(xb), yb)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        model.eval()
        val_loss = sum(F.mse_loss(model(xb), yb).item() for xb, yb in val_loader) / len(val_loader)
        
        if val_loss < best_val:
            best_val = val_loss
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            patience = 0
        else:
            patience += 1
        
        if verbose and epoch % 25 == 0:
            print(f"    Epoch {epoch+1:3d}: train={train_loss:.4f} val={val_loss:.4f}")
        
        if patience >= 30:
            break
    
    model.load_state_dict({k: v.to(device) for k, v in best_state.items()})
    return model

def evaluate(model, loader):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for xb, yb in loader:
            preds.append(model(xb).cpu().numpy())
            targets.append(yb.cpu().numpy())
    p, t = np.concatenate(preds), np.concatenate(targets)
    return {'MSE': mean_squared_error(t, p), 'MAE': mean_absolute_error(t, p)}

# ==============================================================================
# ABLATION
# ==============================================================================
def run_ablation(data, config):
    print("\n" + "="*60)
    print("ABLATION STUDY")
    print("="*60)
    
    configs = {
        'Full': {'use_graph': True, 'use_memory': True, 'use_sharing': True},
        'w/o GGCF': {'use_graph': False, 'use_memory': True, 'use_sharing': True},
        'w/o TMB': {'use_graph': True, 'use_memory': False, 'use_sharing': True},
        'w/o PWS': {'use_graph': True, 'use_memory': True, 'use_sharing': False},
    }
    
    results = []
    loaders = create_loaders(data, config)
    
    for name, flags in configs.items():
        torch.manual_seed(config['SEEDS'][0])
        np.random.seed(config['SEEDS'][0])
        
        model = CausalGraphMemoryNet(
            data['input_dim'], config['SEQ_LEN'], config['PRED_LEN'],
            config['HIDDEN_DIM'], config['N_HEADS'], config['N_LAYERS'],
            config['PATCH_SIZE'], config['N_MEMORY'], config['GRAPH_DIM'],
            config['DROPOUT'], **flags
        )
        model = train_model(model, loaders['train'], loaders['val'], config, verbose=False)
        m = evaluate(model, loaders['test'])
        results.append({'Config': name, 'MSE': m['MSE'], 'MAE': m['MAE']})
        print(f"  {name:12s}: MSE={m['MSE']:.4f}")
    
    df = pd.DataFrame(results)
    full_mse = df[df['Config'] == 'Full']['MSE'].values[0]
    
    print("\nComponent Impact:")
    for _, r in df[df['Config'] != 'Full'].iterrows():
        delta = ((r['MSE'] - full_mse) / full_mse) * 100
        print(f"  {r['Config']:12s}: {delta:+.2f}%")
    
    return df

# ==============================================================================
# MAIN
# ==============================================================================
def run():
    print("="*60)
    print("CausalGraphMemoryNet (CGMN)")
    print("="*60)
    
    # Load data once
    data = create_dataset(load_electricity(), CONFIG['SEQ_LEN'], CONFIG['PRED_LEN'])
    
    print("\nBENCHMARK (3 seeds)")
    print("-"*60)
    
    all_results = []
    
    for seed in CONFIG['SEEDS']:
        print(f"\nSeed {seed}:")
        torch.manual_seed(seed)
        np.random.seed(seed)
        
        loaders = create_loaders(data, CONFIG)
        
        model = CausalGraphMemoryNet(
            data['input_dim'], CONFIG['SEQ_LEN'], CONFIG['PRED_LEN'],
            CONFIG['HIDDEN_DIM'], CONFIG['N_HEADS'], CONFIG['N_LAYERS'],
            CONFIG['PATCH_SIZE'], CONFIG['N_MEMORY'], CONFIG['GRAPH_DIM'],
            CONFIG['DROPOUT']
        )
        
        if seed == CONFIG['SEEDS'][0]:
            print(f"  Params: {sum(p.numel() for p in model.parameters()):,}")
        
        model = train_model(model, loaders['train'], loaders['val'], CONFIG)
        m = evaluate(model, loaders['test'])
        
        imp = ((BASELINE['mse'] - m['MSE']) / BASELINE['mse']) * 100
        print(f"  MSE: {m['MSE']:.4f} ({imp:+.1f}%)")
        
        all_results.append({'Seed': seed, 'MSE': m['MSE'], 'MAE': m['MAE'], 'Improvement': imp})
    
    df = pd.DataFrame(all_results)
    
    print("\n" + "="*60)
    print("RESULTS SUMMARY")
    print("="*60)
    print(f"  MSE: {df['MSE'].mean():.4f} ± {df['MSE'].std():.4f}")
    print(f"  MAE: {df['MAE'].mean():.4f} ± {df['MAE'].std():.4f}")
    print(f"  Improvement: {df['Improvement'].mean():+.1f}% ± {df['Improvement'].std():.1f}%")
    print(f"  Baseline: {BASELINE['mse']:.4f}")
    
    # Ablation
    abl_df = run_ablation(data, CONFIG)
    
    # Model Analysis
    print("\n" + "="*60)
    print("MODEL ANALYSIS")
    print("="*60)
    
    torch.manual_seed(CONFIG['SEEDS'][0])
    model = CausalGraphMemoryNet(
        data['input_dim'], CONFIG['SEQ_LEN'], CONFIG['PRED_LEN'],
        CONFIG['HIDDEN_DIM'], CONFIG['N_HEADS'], CONFIG['N_LAYERS'],
        CONFIG['PATCH_SIZE'], CONFIG['N_MEMORY'], CONFIG['GRAPH_DIM'],
        CONFIG['DROPOUT']
    )
    loaders = create_loaders(data, CONFIG)
    model = train_model(model, loaders['train'], loaders['val'], CONFIG, verbose=False)
    
    stats = model.get_stats()
    print(f"  Parameters: {stats['params']:,}")
    print(f"  Graph sparsity: {stats.get('graph_sparsity', 0)*100:.1f}%")
    print(f"  Memory utilization: {stats.get('memory_util', 0)*100:.1f}%")
    print(f"  Weight sharing: {stats.get('sharing', 0)*100:.1f}%")
    
    return {'benchmark': df, 'ablation': abl_df}

if __name__ == "__main__":
    results = run()

CausalGraphMemoryNet (CGMN)

BENCHMARK (3 seeds)
------------------------------------------------------------

Seed 42:
  Params: 51,353
    Epoch   1: train=1.1290 val=0.9029
    Epoch  26: train=0.0413 val=0.0409
    Epoch  51: train=0.0403 val=0.0401
    Epoch  76: train=0.0399 val=0.0399
  MSE: 0.0417 (+19.1%)

Seed 123:
    Epoch   1: train=1.1007 val=0.8474
    Epoch  26: train=0.0416 val=0.0407
    Epoch  51: train=0.0404 val=0.0403
    Epoch  76: train=0.0400 val=0.0399
  MSE: 0.0418 (+18.8%)

Seed 456:
    Epoch   1: train=1.1291 val=0.8405
    Epoch  26: train=0.0413 val=0.0406
    Epoch  51: train=0.0404 val=0.0410
    Epoch  76: train=0.0399 val=0.0401
  MSE: 0.0418 (+18.8%)

RESULTS SUMMARY
  MSE: 0.0418 ± 0.0001
  MAE: 0.1630 ± 0.0002
  Improvement: +18.9% ± 0.1%
  Baseline: 0.0515

ABLATION STUDY
  Full        : MSE=0.0417
  w/o GGCF    : MSE=0.0417
  w/o TMB     : MSE=0.0419
  w/o PWS     : MSE=0.0417

Component Impact:
  w/o GGCF    : +0.16%
  w/o TMB     : +0.43%
  w/